In [1]:
import os
import pandas as pd
import seaborn as sns
from mlxtend.plotting import plot_learning_curves
import matplotlib.pyplot as plt
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import neighbors
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
import nltk
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")


from nltk.corpus import stopwords
import multiprocessing

cores = multiprocessing.cpu_count()
doc2VecModel = "doc2Vec_model.sav"
LRdoc2VecModel = "LR_doc2Vec_model.sav"

path1 = "../input/ps-dataset/powershell_benign_dataset/"
path2 = "../input/ps-dataset/malicious_pure"
#path3 = r"D:\Test Scripts"
labels = []
text = []
testText = []
newText = []
#new_str = re.sub('[^a-zA-Z0-9\n\.]', ' ', string)
fName= []
regex = r':\s*\n+'
vectorizer = CountVectorizer(stop_words='english', max_features=5000)

def test_bowser(path):
        with open(path, encoding='utf-8') as f:
            t_content = f.read()
            t_content = t_content.lower()
            t_content = t_content.replace("(", "")
            t_content = t_content.replace(")", "")
            t_content = t_content.replace("[", "")
            t_content = t_content.replace("]", "")
            t_content = t_content.replace("$", "")
            t_content = t_content.replace("-", "")
            t_content = t_content.replace("=", "")
            t_content = t_content.replace("", "")
            t_content = t_content.replace("{", "")
            t_content = t_content.replace("}", "")
            t_content = t_content.replace(".", "")
            t_content = t_content.replace("_", "")
            t_content = t_content.replace("|", "")
            t_content = t_content.replace(":", "")
            t_content = t_content.replace("|", "")
            t_content = t_content.replace("/", "")
            t_content = t_content.replace("@", "")
            t_content = t_content.replace("?", "")
            t_content = t_content.replace(",", "")
            t_content = t_content.replace(">", "")
            t_content = t_content.replace("<", "")
            t_content = t_content.replace(";", "")
            t_content = t_content.replace("\\", "")
            t_content = t_content.replace("+", "")
            t_content = t_content.replace("\"","")
            t_content = t_content.replace("\'","")
            t_content = t_content.replace("`", "")
            t_content = t_content.replace("*", "")
            t_content = " ".join(t_content.split())
            newText.append(t_content)
            return t_content

def bowser(path, label):
    for filename in os.listdir(path):
        labels.append(label)
        filename = os.path.join(path, filename)
        with open(filename, encoding='utf-8') as f:
                    content = f.read()
                    content = content.lower()
                    content = content.replace("(", "")
                    content = content.replace(")", "")
                    content = content.replace("[", "")
                    content = content.replace("]", "")
                    content = content.replace("$", "")
                    content = content.replace("-", "")
                    content = content.replace("=", "")
                    content = content.replace("", "")
                    content = content.replace("{", "")
                    content = content.replace("}", "")
                    content = content.replace(".", "")
                    content = content.replace("_", "")
                    content = content.replace("|", "")
                    content = content.replace(":", "")
                    content = content.replace("|", "")
                    content = content.replace("/", "")
                    content = content.replace("@", "")
                    content = content.replace("?", "")
                    content = content.replace(",", "")
                    content = content.replace(">", "")
                    content = content.replace("<", "")
                    content = content.replace(";", "")
                    content = content.replace("\\", "")
                    content = content.replace("+", "")
                    content = content.replace("\"", "")
                    content = content.replace("\'", "")
                    content = content.replace("`", "")
                    content = content.replace("*", "")
                    content = " ".join(content.split())
                    testText.append(content)


#test_bowser(path3)
bowser(path1,"Benign")
bowser(path2,"Malicious")
#print(testText[6])
numFiles = len(testText)

df = pd.DataFrame(testText, index=range(numFiles), columns=["Powershell Code"])
df['Labels'] = labels
#df.to_csv('tester.csv')
print(df)

train, test = train_test_split(df, test_size=0.3, random_state=42)

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens


In [2]:
def bow(data,ngrams =(1,1)):
    vectorizer = CountVectorizer(stop_words='english',max_features=5000)
    train = vectorizer.fit_transform(data)
    
    return train,vectorizer

train_bow, vectorizer = bow(train['Powershell Code'])
test_bow = vectorizer.transform(test['Powershell Code'])

In [3]:
X_train= train_bow
Y_train= train['Labels']
X_test = test_bow
Y_test = test['Labels']

model = LogisticRegression(class_weight="balanced")
model.fit(X_train,Y_train)
st = time.time()
y_pred = model.predict(X_test)
end = time.time()
print(classification_report(Y_test,y_pred,digits=6))
print(end-st)

In [4]:
RF = RandomForestClassifier()
RF.fit(X_train,Y_train)
st = time.time()
y_pred = RF.predict(X_test)
end= time.time()
print(classification_report(Y_test,y_pred,digits=6))
cf_matrix = confusion_matrix(Y_test, y_pred)
print(cf_matrix)
print(end-st)

In [5]:
KNN= KNeighborsClassifier(n_neighbors=5,weights='uniform',n_jobs=1,algorithm="auto",leaf_size=30)
KNN.fit(X_train,Y_train)
st = time.time()
y_pred = KNN.predict(X_test)
end = time.time()
print(classification_report(Y_test,y_pred,digits=6))
print(end-st)

In [6]:
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Random Forrest Classification \n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [7]:
from mlxtend.plotting import plot_learning_curves

svclassifier = SVC(kernel='linear')
plot_learning_curves(X_train, Y_train, X_test, Y_test, svclassifier)
plt.show()

svclassifier.fit(X_train, Y_train)

y_pred = svclassifier.predict(X_test)
print(classification_report(Y_test,y_pred,digits=6))


In [8]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, Y_train)
y_pred = naive_bayes_classifier.predict(X_test)

print(classification_report(Y_test,y_pred,digits=6))


In [9]:
dataset_2 = pd.read_csv('../input/ps-dataset-2/Dataset_2.csv')
train2, test2 = train_test_split(dataset_2, test_size=0.3, random_state=42)


In [10]:
train_bow2, vectorizer2 = bow(train2['Powershell Code'])
test_bow2 = vectorizer2.transform(test2['Powershell Code'])

X_train2= train_bow2
Y_train2= train2['Labels']
X_test2 = test_bow2
Y_test2 = test2['Labels']

In [11]:
RF2 = RandomForestClassifier()
RF2.fit(X_train2,Y_train2)

y_pred = RF2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))
cf_matrix = confusion_matrix(Y_test2, y_pred)
print(cf_matrix)

In [12]:
KNN2= KNeighborsClassifier(n_neighbors=5,weights='uniform',n_jobs=1,algorithm="auto",leaf_size=30)
KNN2.fit(X_train2,Y_train2)
st = time.time()
y_pred = KNN2.predict(X_test2)
end = time.time()
print(classification_report(Y_test2,y_pred,digits=6))
print(end-st)

In [13]:
LR2 = LogisticRegression(class_weight="balanced")
LR2.fit(X_train2,Y_train2)


y_pred = LR2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [14]:
naive_bayes_classifier2 = MultinomialNB()
naive_bayes_classifier2.fit(X_train2, Y_train2)
y_pred = naive_bayes_classifier2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))


In [15]:
from mlxtend.plotting import plot_learning_curves

svclassifier2 = SVC(kernel='linear')
plot_learning_curves(X_train2, Y_train2, X_test2, Y_test2, svclassifier2)
plt.show()

svclassifier2.fit(X_train2, Y_train2)

y_pred = svclassifier2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [16]:
#SVM CROSS VAL 
train_bow2, vectorizer = bow(train['Powershell Code'])
test_bow3 = vectorizer.transform(test['Powershell Code'])
X_test = test_bow3
Y_test = test['Labels']
y_pred = svclassifier2.predict(X_test)
print(classification_report(Y_test,y_pred,digits=6))

In [17]:
y_pred = naive_bayes_classifier2.predict(X_test)
print(classification_report(Y_test,y_pred,digits=6))

In [18]:
y_pred = RF2.predict(X_test)
print(classification_report(Y_test,y_pred,digits=6))

In [19]:
y_pred = KNN2.predict(X_test)
print(classification_report(Y_test,y_pred,digits=6))

In [20]:
y_pred = LR2.predict(X_test)
print(classification_report(Y_test,y_pred,digits=6))
plot_learning_curves(X_train2, Y_train2, X_test, Y_test, LR2)

In [21]:
#--------------CROSS VALIDATION OF MODEL 1 ON DATASET2-------------#
#SVM
train_bow2, vectorizer = bow(train2['Powershell Code'])
test_bow2 = vectorizer.transform(test2['Powershell Code'])
X_test2 = test_bow2
Y_test2 = test2['Labels']
y_pred = svclassifier.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [22]:
y_pred = naive_bayes_classifier.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [23]:
y_pred = model.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [24]:
y_pred = KNN.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [25]:
y_pred = RF.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))